# Mental Health Classifier with Explainability and Fairness Analysis

## Title Slide
- **Title**: Mental Health Classifier with Explainability and Fairness Analysis
- **Team Members**: Sameer Siddhant

In [ ]:
# Install required packages
!pip install --upgrade transformers

# Import necessary libraries
import transformers  # Import the library

import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import AdamW

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Define a custom scheduler (alternative to get_scheduler)
def linear_scheduler_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda)

# Example setup for model and optimizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Example scheduler usage
num_training_steps = 1000
num_warmup_steps = 100
scheduler = linear_scheduler_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# The rest of your code (e.g., dataset preparation, training loop) can now use this scheduler.


In [ ]:
import pandas as pd
import re
from transformers import AutoTokenizer

# Step 1: Load the dataset
data = pd.read_csv(r'C:\Users\91940\Downloads\mental_health (1).csv')

# Step 2: Define a cleaning function
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r"@\w+|\#", '', text)
    text = text.lower()
    return text

# Step 3: Clean the text
data['cleaned_text'] = data['text'].apply(clean_text)

# Step 4: Encode the gender column
data['gender'] = data['gender'].map({'male': 1, 'Female': 0})  # Encode gender

# Step 5: Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Step 6: Tokenize the cleaned text
data['tokens'] = data['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)


In [ ]:
# Step 3: Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)




In [ ]:
# Define the Dataset class
class TextDataset(Dataset):
    def __init__(self, df):
        self.labels = df['label'].values
        self.genders = df['gender'].values
        self.inputs = list(df['tokens'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.inputs[idx], dtype=torch.long),
            'label': torch.tensor(self.labels[idx], dtype=torch.float),
            'gender': torch.tensor(self.genders[idx], dtype=torch.float)
        }

train_dataset = TextDataset(train_data)
test_dataset = TextDataset(test_data)


In [ ]:
# Step 5: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)



In [ ]:
# Step 6: Initialize the model and optimizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)



In [ ]:
# Step 7: Define the loss function and scheduler
criterion = nn.CrossEntropyLoss()
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)



In [ ]:
# Training loop
def train_model():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['input_ids'] > 0, labels=batch['label'].long())
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)



In [ ]:
# Validation function
def validate_model():
    model.eval()
    predictions, true_labels, genders = [], [], []
    with torch.no_grad():
        for batch in test_loader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['input_ids'] > 0)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, axis=-1).tolist())
            true_labels.extend(batch['label'].tolist())
            genders.extend(batch['gender'].tolist())
    return predictions, true_labels, genders



In [ ]:
# Training and evaluation
for epoch in range(3):
    train_loss = train_model()
    predictions, true_labels, genders = validate_model()
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Accuracy = {accuracy:.4f}, F1 Score = {f1:.4f}")



In [ ]:
# Fairness Analysis
def fairness_analysis(predictions, true_labels, genders):
    results = []
    for gender in [0, 1]:
        indices = [i for i, g in enumerate(genders) if g == gender]
        gender_predictions = [predictions[i] for i in indices]
        gender_true_labels = [true_labels[i] for i in indices]
        acc = accuracy_score(gender_true_labels, gender_predictions)
        f1 = f1_score(gender_true_labels, gender_predictions, average='weighted')
        results.append({"Gender": "Female" if gender == 0 else "Male", "Accuracy": acc, "F1 Score": f1})
    return pd.DataFrame(results)

fairness_results = fairness_analysis(predictions, true_labels, genders)
print("Fairness Analysis Results:")
print(fairness_results)



In [ ]:
# Visualize fairness analysis
sns.barplot(x="Gender", y="Accuracy", data=fairness_results)
plt.title("Fairness Analysis - Accuracy by Gender")
plt.show()

sns.barplot(x="Gender", y="F1 Score", data=fairness_results)
plt.title("Fairness Analysis - F1 Score by Gender")
plt.show()



In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# Explainability using SHAP
explainer = shap.Explainer(model, tokenizer)
sample_texts = test_data['cleaned_text'].iloc[:10].tolist()  # Take a small sample for explainability
sample_tokens = tokenizer(sample_texts, truncation=True, padding=True, return_tensors="pt", max_length=128)
shap_values = explainer(sample_tokens)



In [ ]:
# Visualize SHAP values
for i in range(len(sample_texts)):
    print(f"Explanation for text {i + 1}: {sample_texts[i]}")
    shap.plots.text(shap_values[i])



In [ ]:
# Save the model for deployment
model.save_pretrained("mental_health_model")
tokenizer.save_pretrained("mental_health_model")